# Mathematical Overview

In this section we present a mathematical overview of using Linear Predictive Coding (LPC) to extract spectral envelopes for cross-synthesis.

Before explaining LPC, we must first cover some necessary background: the mathematical representations of filters, and the spectral analysis of noise. 



# Filters 

A *Linear Time Invariant (LTI)* filter can be represented in the time-domain as an impulse-response function $h$ that convolves an input signal $x$ to produce a filtered output signal $y$.

$$
y(n) = (h * x)(n)
$$

By the Convolution Theorem this corresponds to windowing in the frequency domain, where $H(k)$ is called the *transfer function* of the filter:

$$
Y(k) = H(k)X(k) \implies H(k) = \frac{Y(k)}{X(k)}
$$

An alternative model represents filters as computes an output signal $y(n)$ from a linear combination of past input and output samples:

$$
\begin{align}
y(n) &= b_0x(n) + b_1x(n-1) + ... + b_Mx(n-N) \\ 
     & \          - a_1y(n-1) - ... - a_Ny(n-M) \\
     &= \sum_{i=0}^{N}b_ix(n-i) - \sum_{j=1}^{M}a_jy(n-j)
\end{align}
$$

This representation is called the *Difference Equation* of the filter, with ${b}_{i=0}^N$ feedfoward coefficients and ${a}_{j=1}^M$ feedback coefficients. 

__Note: $i$ and $j$ here function as indices, not as the complex value $\sqrt{-1}$__!

Let $\omega = e^{-\sqrt{-1}(2\pi)/N}$. Taking the DFT of the difference equation, we find

\begin{align}
Y(k) &= DFT[\sum_{i=0}^{N}b_ix(n-i) - \sum_{j=1}^{M}a_jy(n-j)] \\
&= DFT[\sum_{i=0}^{N}b_ix(n-i)] - DFT[\sum_{j=1}^{M}a_jy(n-j)] \\
&= \sum_{i=0}^{N}b_iDFT[SHIFT_i\{x(n)\}] - \sum_{j=1}^{M}a_jDFT[SHIFT_j\{y(n)\}] \\
&= \sum_{i=0}^{N}b_i\omega^iX(k) - \sum_{j=1}^{M}a_j\omega^jY(k) \\
&= X(k)\underbrace{\sum_{i=0}^{N}b_i\omega^i}_{DFT[b]} - Y(k)\sum_{j=1}^{M}a_j\omega^j \\
\end{align}

Rearranging, we have

\begin{aligned}
& Y(k)\underbrace{[1 + \sum_{j=1}^{M}a_j\omega^j]}_{DFT[a]} = X(k)\underbrace{\sum_{i=0}^{N}b_i\omega^i}_{DFT[b]} \\
& \implies Y(k)A(k) = X(k)B(k) \\
& \implies H(k) = \frac{Y(k)}{X(k)} = \frac{B(k)}{A(k)} \\
\end{aligned}

Hence the transfer function $H$ of an LTI filter may also be represented as the quotient of its feedfoward and feedback coefficients in the frequency domain. This representation will be relevant to understanding LPC as computing a recursive, all-pole filter.


# Spectrum Analysis of Noise

A statistical interpretation of LPC assumes all time-domain signals can be modeled as outputs from an all-pole filter driven by white noise. (Note all-pole means the difference equation of the filter has 0 feedfoward coefficients, and only feedback coefficients). 

In statistical signal processing, *white noise* is modeled as a *stationary stochastic process*, i.e. a sequence of random variables all drawn from the same, static probability distribution. Given such a noise sequence $x \in \mathbb{R}^N$, we assume the distribution has $\mu_x = 0$ and variance $\sigma_x^2$. 

What are the spectral characteristics of this noise sequence? It will help to recall the definition of the *Power Spectral Density* function

$$
S_x(k) = DFT_k[r_x]
$$

Where $r_x$ is the *autocorrelation function*

$$
r_x(m) = \frac{1}{N}(x \otimes x)(m) = \frac{1}{N}\sum_{n=0}^{N-1}x(n)x(n+m)
$$

For a white-noise signal $x$ with $\mu_x = 0$, note that as $N \to \infty$, 

$$
r_x(0) = \frac{1}{N}\sum_{n=0}^{N-1}|x(n)|^2 = \sigma_x^2
$$

And for $m \neq 0$, the expected value $E[r_x(m)] \to 0$

$$
\begin{align}
E[r_x(m)] &= E[\frac{1}{N}\sum_{n=0}^{N-1}x(n)x(n+m)] \\
&= \frac{1}{N}\sum_{n=0}^{N-1}E[x(n)]E[x(n+m)] \\
&= \frac{1}{N}\sum_{n=0}^{N-1}\mu_x^2 \\
&= 0
\end{align}
$$

The autocorrelation function $r_x(m)$ of a white noise signal $x(n)$ is therefore all 0s except at index 0 where $r_x(0) = \sigma_x^2$. In otherwords, the autocorrelation of white noise (averaged over infinite samples) is equivalent to the weighted impulse function $r_x = \sigma_x^2\delta$.

The Power Spectral Density of white noise is therefore proportional to the DFT of the impulse function, which gives a uniform frequency response across all frequency bins.

$$
S_x(k) = DFT_{k}[r_x] = DFT_k[\sigma_x^2\delta] = \sigma_x^2
$$

Hence we say the average magnitude spectrum of white noise is *flat*.


# Linear Prediction Coding

Now we are ready to analyze LPC. Given an input signal $y \in \mathbb{R}^N$, linear prediction aims to find a set of *prediction coefficients* $\{a\}_{i=1}^M$ such that $\forall n < N, y(n)$ can be accurately estimated as a linear combination of the previous $M$ samples. I.e.

$$
y(n) = e(n) - \sum_{i=1}^{M}a_iy(n-i)
$$

Here $M$ denotes the *order* of the linear predictor, and $e(n)$ denotes the *prediction error* at sample $n$. Computing this linear predictor requires solving an optimization problem: what are the optimal $\{a\}_{i=1}^M$ which minimize the sum of squared errors $||e||^2$? 

Before delving into a solution, note how similar the above linear equation looks to the difference equations discussed earlier! Like we did with difference equations, let's take the DFT of both sides:

\begin{align}
Y(k) &= DFT[e] - DFT[\sum_{i=1}^{M}a_iy(n-i)] \\
&= E(k) - \sum_{i=1}^{M}a_iDFT[SHIFT_i\{y(n)\}] \\
&= E(k) - Y(k)\sum_{i=1}^{M}a_i\omega^i 
\end{align}

Rearranging terms and factoring out $Y(k)$, we have

$$
Y(k)\underbrace{[1 + \sum_{i=1}^{M}a_i\omega^i]}_{A(k)} = E(k) \implies Y(k) = \frac{E(k)}{A(k)} \implies \frac{1}{A(k)} = \frac{Y(k)}{E(k)}
$$

This last equation provides remarkable insight into what exactly LPC is doing, and how it is managing to extract the spectral envelope of the input signal $y$. The prediction coefficients found by LPC correspond to a designing a filter with impulse response $h = \{1, a_1, a_2, ..., a_m\}$ and transfer function $H(k) = 1/A(k)$ that maps the error spectrum $E(k)$ to the signal spectrum $Y(k)$. Minimizing $||e||^2$ corresponds to maximally flattening $||E||$, and as discussed above, flat spectra indicate the underlying signal can be modeled as white noise. Once $||E||$ has been optimized, $h$ will function as a noise-driven recursive filter that approximately reconstructs the original signal $y$! And because $|E(k)|$ is flat for white noise, the filter's frequency response $|A(k)|$ must also approximate $|Y(k)|$. 

$$
\begin{aligned}
& (h * e) \approx y & H(k)E(k) \approx Y(k) \\
\end{aligned}
$$

It makes sense that $e(n)$ can be modeled as white noise under optimal prediction coefficients because it represents all new, unpredictable information entering the signal at sample $n$. Furthermore, if $||e||^2$ is minimized, all $e(i)$ should be independent, with no correlation to one another. Let us therefore assume the prediction error can be modeled as white noise with $\mu_e = 0$ and variance $\sigma_e^2$.

Recalling that by the Correlation theorem, $y \otimes y \longleftrightarrow |Y(k)|^2$ we can now formally define the spectral envelope of $y$:

$$ 
\begin{align}
S_y(k) &= DFT[\frac{1}{N}(y \otimes y)] \\
& = \frac{1}{N}|Y(k)|^2 = \frac{|E(k)|^2/N}{|A(k)|^2} \\
&= \frac{S_e(k)}{|A(k)|^2} = \frac{\sigma_e^2}{|A(k)|^2} \\
\text{EnvelopeLPC}_y(k) &= \sqrt{S_y(k)} = \frac{\sigma_e}{|A(k)|}
\end{align}
$$

In words, the spectral envelop found by LPC is inverse magnitude spectrum of the prediction filter $A(k)$.




### Solving for Prediction Coefficients

Let $\{\hat{a}\}_i^M$ denote the optimal prediction coefficients which minimize $||e||^2$. We can find these values by taking the partial derivative of the *cost function* $J(a) : \mathbb{R}^M \to \mathbb{R}$ with respsect to each $a_i$, and setting it equal to 0. This 0-point represents a minimum in the cost function along the space $a_i=\hat{a_i}$. Solving this set of equations corresponds to finding the values of each $a_i$ which minimize $||e||^2$ 

\begin{align}
J(a) &= ||e^2|| = \sum_{n=0}^{N-1}[y(n) + \sum_{i=1}^{M}a_iy(n-i)]^2 \\
\frac{\partial J}{\partial a_k} &= 2\sum_{n=0}^{N-1}[y(n) + \sum_{i=1}^{M}a_iy(n-i)]y(n-k) \\
\end{align}

Rearranging terms to solve for $\partial J / \partial a_k = 0$, we have

\begin{align}
-\sum_{n=0}^{N-1}y(n)y(n-k) &= \sum_{n=0}^{N-1}\sum_{i=1}^{M}\hat{a_i}y(n-i)y(n-k) \\
&= \sum_{i=1}^{M}\hat{a_i}\sum_{n=0}^{N-1}y(n-i)y(n-k)
\end{align}

The above equality can now be expressed in terms of the Bartlett-windowed biased acyclic autocorrelation function $\gamma$:

$$
\gamma_y(k) = \sum_{n}y(n)y(n+k)
$$

Note that for real signals this autocorrelation is a real, even function, i.e. $\gamma(k) =  \gamma(-k) \in \mathbb{R}$ Substituting, we now have

$$
-\gamma_y(k) = \sum_{i=1}^M\hat{a_i}\gamma_y(|k-i|)
$$

Because we are taking the partial derivative of $J(a)$ with respect to each $a_k$, we end up with a system of $M$ linear equations and $M$ unknowns, which may be represented as a matrix multiplication of matrix $R \in \mathbb{R}^{M \times M}$ and $p \in \mathbb{R}^M$:

$$
R\hat{a}= p
$$

Here $R(i,k) = \gamma_y(|k-i|)$ and $p(k) = -\gamma_y(k)$. The optimal prediction coefficients $\hat{a}$ can then be found by:

$$
\hat{a} = R^{-1}p
$$

Note that the covariance matrix $R$ is symmetric and *Toeplitz*, and therefore there exists an $O(M^2)$ solution to the above system of linear equations via the *Levinson-Durbin Algorithm*. Dicussing this algorithm goes beyond the scope of our project. In practice, we often have very small $M$, e.g. $M<10$ when modeling human speech, and so using a built-in linear equation solver such as `np.linalg.lstsq` is perfectly functional.

Lastly, we can extract the spectral envelope of $y$ by computing:

$$
\text{EnvelopeLPC}_y(k) = \frac{1}{|DFT_k(\hat{a})|} = \frac{1}{|\hat{A}(k)|}
$$

### Why LPC envelopes are "good"

We can give some brief mathematical intuition behind why LPC is a "good" envelop extractor as follows. Let $\hat{Y}$ denote our *estimated* signal output and $\hat{A}$ our estimated prediction coefficients. By Rayleigh's Energy theorem,

\begin{align}
\sum_{n=1}^N|e(n)|^2 &= \frac{1}{N}\sum_{k=1}^N|E(k)|^2 \\
&= \frac{1}{N}\sum_{k=1}^N|A(k)Y(k)|^2 \\
&= \frac{1}{N}\sum_{k=1}^N|\frac{\sigma_e^2}{\hat{Y}(k)}Y(k)|^2 \\
&= \frac{\sigma_e^2}{N}\sum_{k=1}^N|\frac{Y(k)}{\hat{Y}(k)}|^2 \\
\end{align}

We can see that $|e(n)|^2$ is minimized when $\hat{Y}(k) > Y(k)$ i.e. when the estimated spectral magnitude exceeds the signal spectral magnitude. This means that LPC prioritizes tracking peaks over valleys, making for more accurate envelope extraction.

## References

https://ccrma.stanford.edu/~jos/sasp/Spectrum_Analysis_Noise.html

https://ccrma.stanford.edu/~jos/sasp/Cross_Synthesis.html

https://ccrma.stanford.edu/~jos/filters/Z_Transform_Difference_Equations.html
